# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=3158056389
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [4]:
sgmt

,Name,Time,Speed,Power,Info
0,Saferbo - Sofasa 4.56km 78m -2%,6:51,40.0km/h,367W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
1,Mayorca - Pandequeso 1.82km 19m -1%,3:33,30.9km/h,300W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
2,Peldar - Mayorca (Envigado) 1.63km 19m 1%,4:28,21.9km/h,271W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
3,Semáforo Sofasa - Semáforo Kia 0.78km 8m 1%,2:05,22.7km/h,293W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
4,Semáforo Kia - Bomba Esso 0.63km 10m 2%,1:46,21.5km/h,262W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
5,Glorieta Mayorca - Bomba Terpel 0.75km 8m 1%,2:26,18.6km/h,211W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
6,semaforo Mayorca- Semaforo zona Norte 1 0.46k...,1:20,20.9km/h,228W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
7,Semaforo zona Norte 2 - Semaforo cruce 0.37km...,1:02,21.6km/h,250W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
8,Loma Colegio Adelaida Correa 0.60km 96m 15%,8:01,4.5km/h,184W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."
9,Regional Estrella - Envigado 4.86km 79m -2%,7:29,39.0km/h,359W,"{'Calories': {'Avg': '858', 'Max': '858'}, 'El..."


In [5]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [6]:
df.to_json(db_file)

### Save notebook here

In [7]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [8]:
!bash commit.sh

[master 0bb8f01] Evening Ride 7:11 PM on Wednesday, August 26, 2020
 2 files changed, 32 insertions(+), 131 deletions(-)
 rewrite db/strava.json (79%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 7.31 KiB | 1.04 MiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:restrepo/strava.git
   8ebac56..0bb8f01  master -> master


In [9]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? y
